# Hyperparameter tuning using Oputa on XGB model and LGBM model

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm

from sklearn.metrics.pairwise import rbf_kernel

from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline, make_pipeline, FunctionTransformer
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline
import xgboost as xgb
import warnings
warnings.simplefilter('ignore')

In [3]:
agg_train = pd.read_csv("C:/Users/Asus/Documents/MMAI/869_MachineLearningAI/Team_assignment/train/agg_train.csv",low_memory=False)
agg_test = pd.read_csv("C:/Users/Asus/Documents/MMAI/869_MachineLearningAI/Team_assignment/test/agg_test.csv",low_memory=False)

In [4]:
client_train =pd.read_csv("C:/Users/Asus/Documents/MMAI/869_MachineLearningAI/Team_assignment/train/client_train.csv",low_memory=False)
target = client_train['target']

## Tuning LGBM model

In [16]:
import optuna
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

def objective(trial,X,y):
 # 'is_unbalance':True,
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt', 
        'metric': 'auc',
        "n_estimators": 1000,
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "num_iterations": 500,
        "num_leaves": trial.suggest_int('num_leaves',20,100),
        "min_child_samples": trial.suggest_int("min_child_samples",20,500),
        "max_depth": trial.suggest_int("max_depth",5,20),
        "subsample": trial.suggest_float("subsample",0.5,1.0),
        "feature_fraction": trial.suggest_float("feature_fraction",0.5,1.0)
    }
    
    light_GBM =ImPipeline([
    ('preprocessing', preprocessing),
    ('model', LGBMClassifier(**params))
    ])
    
    # Fit the model
    light_GBM.fit(agg_train, target)
    cv_scores = cross_val_score(light_GBM, X, y, cv=10, n_jobs=-1, scoring="roc_auc")
    score = np.mean(cv_scores)
    # scorestd = cv_scores.std()
    return score  # Replace with appropriate metric

# Create a study object and specify the direction is 'maximize'.
study = optuna.create_study(direction='maximize')

# Start the optimization
study.optimize(lambda trial: objective(trial, agg_train, target), n_trials=100,  gc_after_trial=True)

# Print the optimal parameters
print(study.best_params)

[I 2023-12-09 18:37:45,562] A new study created in memory with name: no-name-99621350-0d89-4e5b-a608-9682082cd8ba


[LightGBM] [Warning] feature_fraction is set=0.7028918851524675, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7028918851524675
[LightGBM] [Warning] feature_fraction is set=0.7028918851524675, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7028918851524675
[LightGBM] [Info] Number of positive: 7566, number of negative: 127927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3626
[LightGBM] [Info] Number of data points in the train set: 135493, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.055841 -> initscore=-2.827795
[LightGBM] [Info] Start training from score -2.827795


[I 2023-12-09 18:38:35,343] Trial 0 finished with value: 0.8356320075835908 and parameters: {'scale_pos_weight': 2, 'learning_rate': 0.003235804348291526, 'num_leaves': 71, 'min_child_samples': 200, 'max_depth': 13, 'subsample': 0.7680476365147786, 'feature_fraction': 0.7028918851524675}. Best is trial 0 with value: 0.8356320075835908.


[LightGBM] [Warning] feature_fraction is set=0.9445463776045844, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9445463776045844
[LightGBM] [Warning] feature_fraction is set=0.9445463776045844, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9445463776045844
[LightGBM] [Info] Number of positive: 7566, number of negative: 127927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005758 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3626
[LightGBM] [Info] Number of data points in the train set: 135493, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.055841 -> initscore=-2.827795
[LightGBM] [Info] Start training from score -2.827795
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2023-12-09 18:39:20,867] Trial 1 finished with value: 0.837212731490806 and parameters: {'scale_pos_weight': 3, 'learning_rate': 0.01646756910707209, 'num_leaves': 88, 'min_child_samples': 159, 'max_depth': 19, 'subsample': 0.7084242269899717, 'feature_fraction': 0.9445463776045844}. Best is trial 1 with value: 0.837212731490806.


...

[LightGBM] [Warning] feature_fraction is set=0.5944872569418991, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5944872569418991
[LightGBM] [Warning] feature_fraction is set=0.5944872569418991, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5944872569418991
[LightGBM] [Info] Number of positive: 7566, number of negative: 127927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3626
[LightGBM] [Info] Number of data points in the train set: 135493, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.055841 -> initscore=-2.827795
[LightGBM] [Info] Start training from score -2.827795


[I 2023-12-09 19:49:27,963] Trial 98 finished with value: 0.8388143249938416 and parameters: {'scale_pos_weight': 5, 'learning_rate': 0.012629129326199062, 'num_leaves': 39, 'min_child_samples': 267, 'max_depth': 15, 'subsample': 0.9951298673503509, 'feature_fraction': 0.5944872569418991}. Best is trial 88 with value: 0.839493765558394.


[LightGBM] [Warning] feature_fraction is set=0.5499266726468366, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5499266726468366
[LightGBM] [Warning] feature_fraction is set=0.5499266726468366, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5499266726468366
[LightGBM] [Info] Number of positive: 7566, number of negative: 127927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004285 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3626
[LightGBM] [Info] Number of data points in the train set: 135493, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.055841 -> initscore=-2.827795
[LightGBM] [Info] Start training from score -2.827795


[I 2023-12-09 19:50:00,656] Trial 99 finished with value: 0.83805458547897 and parameters: {'scale_pos_weight': 5, 'learning_rate': 0.015023117821750804, 'num_leaves': 26, 'min_child_samples': 179, 'max_depth': 16, 'subsample': 0.9624321407216059, 'feature_fraction': 0.5499266726468366}. Best is trial 88 with value: 0.839493765558394.


{'scale_pos_weight': 3, 'learning_rate': 0.023458592474959277, 'num_leaves': 43, 'min_child_samples': 123, 'max_depth': 16, 'subsample': 0.9637220440391915, 'feature_fraction': 0.566261327139611}


In [17]:
study.best_params

'''
10 mins
Best is trial 7 with value: 0.8369938153532145.
{'scale_pos_weight': 5, 'learning_rate': 0.007708305735559817}
100 trial 72 mins
Best is trial 88 with value: 0.839493765558394.
{'scale_pos_weight': 3, 'learning_rate': 0.023458592474959277, 'num_leaves': 43, 'min_child_samples': 123, 
'max_depth': 16, 'subsample': 0.9637220440391915, 'feature_fraction': 0.566261327139611}

'''

"\n\nBest is trial 7 with value: 0.8369938153532145.\n{'scale_pos_weight': 5, 'learning_rate': 0.007708305735559817}\nBest is trial 88 with value: 0.839493765558394.\n{'scale_pos_weight': 3, 'learning_rate': 0.023458592474959277, 'num_leaves': 43, 'min_child_samples': 123, \n'max_depth': 16, 'subsample': 0.9637220440391915, 'feature_fraction': 0.566261327139611}\n\n"

In [ ]:
clf_rf = ImPipeline([
        ('preprocessing', preprocessing),
        # ('smote', SMOTE(sampling_strategy=0.3, random_state=42)),  # Adjust as needed
        ('model', RandomForestClassifier(n_estimators=500,
                                            #  **study.best_params,
                                            max_samples= 0.605199046299782, min_samples_split= 60,
                                            class_weight='balanced',
                                             n_jobs=-1,
                                             random_state=42))
    ])
clf_rf.fit(agg_train, target)
fraud_predictions = clf_rf.predict_proba(agg_test)


fraud_predictions = fraud_predictions[:, 1]

In [ ]:
pred = model.predict_proba(agg_test)
preds = pd.DataFrame(pred, columns=['target','target2'])

In [ ]:
preds = pd.DataFrame(preds, columns=['target2'])

In [ ]:
preds.head()

In [ ]:
client_test = pd.read_csv("C:/Users/Asus/Documents/MMAI/869_MachineLearningAI/Team_assignment/test/client_test.csv",low_memory=False)

In [ ]:
submission = pd.DataFrame(
    {
        'client_id': client_test['client_id'],
        'target': preds['target2']
    }
)

submission.head()